# Anillo Obsidian Source Geochemistry
Welcome to the geochemical analysis notebook for an XRF analysis of samples from the Anillo Obsidian in Arequipa, Peru. 

In [ ]:
%pip -q install "plotly==5.18.0" "anywidget==0.9.13" "ipywidgets==8.1.2" "jupyterlab_widgets==3.0.10" "widgetsnbextension==4.0.10"

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from pathlib import Path
from IPython.display import Markdown, display, Javascript
from ipywidgets import Button, Output, VBox, HTML, widgets


In [ ]:
# JavaScript to hide inputs/outputs based on cell tags in order to declutter the notebook

display(HTML("""
<script>
(function() {
  function hideTaggedInputs(){
    // Find all cells
    var cells = document.querySelectorAll('.cell');
    cells.forEach(function(cell){
      var tagEl = cell.querySelector('.celltags, .jp-Cell-tags');
      // In classic notebook, tags aren't always in DOM; so use metadata if available:
    });

    // Classic Notebook: use Jupyter API if available
    if (window.Jupyter && Jupyter.notebook) {
      Jupyter.notebook.get_cells().forEach(function(c) {
        var tags = (c.metadata && c.metadata.tags) ? c.metadata.tags : [];
        if (tags.includes('hide_input')) {
          c.element.find('.input').hide();
        }
        if (tags.includes('hide_output')) {
          c.element.find('.output').hide();
        }
      });
    }
  }

  // Run once now, then again shortly after load (Binder can be slow)
  hideTaggedInputs();
  setTimeout(hideTaggedInputs, 800);
})();
</script>
"""))

## Loading tables from Google Drive
### Designed for the Google Sheet to have three tabs:
*  **KRA21_sources** tab contains source sample geochemistry and Group variable (follow example provided)
*  **srcs_locs** tab contains Latitude and Longitude coordinates for each source
*  **study samples** tab contains geochemistry of samples under study

In [ ]:
# Configuration
USE_GOOGLE_SHEETS = False # Toggle between Google Sheets and local CSV
DATA_DIR = Path("../data")
SHEET_ID = "1R4PlMACBn0l8ZguwtYDlZLnbvORzH5CHhKGFBezjSvk"

## Data Loading

In [ ]:
def get_df(source, sheet_name=None):
    """
    Load dataframe from Google Sheets or local CSV.
    
    Parameters
    ----------
    source : str
        Sheet ID (if USE_GOOGLE_SHEETS=True) or CSV filename
    sheet_name : str, optional
        Required when USE_GOOGLE_SHEETS=True
    """
    if USE_GOOGLE_SHEETS:
        if sheet_name is None:
            raise ValueError("sheet_name required when USE_GOOGLE_SHEETS=True")
        url = (
            f"https://docs.google.com/spreadsheets/d/{source}/gviz/tq"
            f"?tqx=out:csv&sheet={sheet_name}"
        )
        return pd.read_csv(url)
    else:
        path = DATA_DIR / source
        if not path.exists():
            raise FileNotFoundError(f"Missing local file: {path.resolve()}")
        return pd.read_csv(path)

# Load datasets
if USE_GOOGLE_SHEETS:
    srcs = get_df(SHEET_ID, "KRA21_sources")
    srcs_locs = get_df(SHEET_ID, "source_coords")
    study = get_df(SHEET_ID, "samples")
else:
    srcs = get_df("KRA21_sources.csv")
    srcs_locs = get_df("source_coords.csv")
    study = get_df("study_samples.csv")


### Dataset headers are cleaned up (if necessary) ###

In [ ]:
def clean_geochem_df(df):
    """Clean geochemistry dataframe headers and types."""
    # Remove Bruker artifacts and spaces
    df.columns = df.columns.str.replace(r'(Ka1|La1|\s+)', '', regex=True)
    
    # String columns
    string_cols = ['Group', 'Sample', 'Name']
    present_strings = [c for c in string_cols if c in df.columns]
    numeric_cols = df.columns.difference(present_strings)
    
    # Set dtypes
    df[present_strings] = df[present_strings].astype('string')
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Drop all-NaN columns
    df.dropna(axis=1, how='all', inplace=True)
    
    return df

srcs = clean_geochem_df(srcs)
study = clean_geochem_df(study)

# Debug: print dataset headers to confirm header names. 
# print('Dataset Headers')
# print('Study:', study.columns.tolist())
# print('Sources:', srcs.columns.tolist())


In [ ]:
# Data Schema enforcement
# Define required columns for each dataset

REQUIRED = {
    "srcs": ["Sample", "Group", "Rb", "Sr", "Zr"],
    "study": ["Sample", "Group", "Rb", "Sr", "Zr"]
}
# These are minimal for this code to make sense; more elements can be present

def check_schema(df, required, name):
    missing = set(required) - set(df.columns)
    if missing:
        raise ValueError(f"{name} missing columns: {missing}")

check_schema(srcs, REQUIRED["srcs"], "Sources")
check_schema(study, REQUIRED["study"], "Study")

In [ ]:
# The study data table

print('Complete Study Sample Table used in this visualization:')
display(study)

## Use Lasso tool to select obsidian sources from region of interest

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# --- Load coords table ---
url = "https://raw.githubusercontent.com/arf-berkeley/obsidian-source-anillo/main/data/source_coords.csv"
srcs_locs = pd.read_csv(url)

# --- Validate expected columns ---
required = {"Name", "Lat", "Long"}
missing = required - set(srcs_locs.columns)
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}. Found: {list(srcs_locs.columns)}")

# --- Center/zoom to data ---
lat_min, lat_max = srcs_locs["Lat"].min(), srcs_locs["Lat"].max()
lon_min, lon_max = srcs_locs["Long"].min(), srcs_locs["Long"].max()
center_lat, center_lon = (lat_min + lat_max) / 2, (lon_min + lon_max) / 2

max_span = max(lat_max - lat_min, lon_max - lon_min)
zoom = 5 if max_span > 10 else 6 if max_span > 5 else 7 if max_span > 2 else 8 if max_span > 1 else 9 if max_span > 0.5 else 10

names = srcs_locs["Name"].astype(str).tolist()

# --- IMPORTANT: FigureWidget enables Python<->browser selection syncing ---
fig = go.FigureWidget()

fig.add_scattermapbox(
    lat=srcs_locs["Lat"],
    lon=srcs_locs["Long"],
    mode="markers+text",
    text=names,
    textposition="top center",
    textfont=dict(size=11, color="black", family="Arial Black"),
    marker=dict(size=10),
    customdata=names,
    hovertext=names,
    selected=dict(marker=dict(size=14, color="red")),
    unselected=dict(marker=dict(opacity=0.35)),
)

fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom,
    ),
    dragmode="lasso",            # lasso select by default
    clickmode="event+select",
    height=650,
    margin=dict(r=0, l=0, t=0, b=0),
    showlegend=False,
)

# This list will be available later in your notebook
selected_names = []

status = widgets.HTML(
    "<div style='padding:8px; background:#e3f2fd; border-left:4px solid #2196F3;'>"
    "<b>Instructions:</b> Use lasso/box select on the map. Selection is saved to <code>selected_names</code>.</div>"
)

def _on_select(trace, points, selector):
    global selected_names
    inds = list(points.point_inds) if points and points.point_inds is not None else []
    selected_names = [names[i] for i in inds]

    if selected_names:
        status.value = (
            "<div style='padding:8px; background:#c8e6c9; border-left:4px solid #4CAF50;'>"
            f"<b>Captured:</b> {len(selected_names)} selected<br>"
            f"{', '.join(selected_names)}</div>"
        )
    else:
        status.value = (
            "<div style='padding:8px; background:#fff3e0; border-left:4px solid #FF9800;'>"
            "<b>No points selected.</b></div>"
        )

# Attach selection callback
fig.data[0].on_selection(_on_select)

display(widgets.VBox([status, fig]))

In [ ]:
import plotly, sys
print("python:", sys.version)
print("plotly:", plotly.__version__)

## Select Obsidian Sources before proceeding
Use the map above this text to select obsidian sources in your region of interest, then click in the cell below and continue running the notebook.

In [ ]:
# Display results
if 'selected_names' in dir() and selected_names:
    display(HTML(f"""
    <div style="padding: 10px; background-color: #c8e6c9; border-radius: 5px; border-left: 4px solid #4CAF50;">
        <b>✅ Success!</b> Selected <b>{len(selected_names)}</b> source(s)
    </div>
    """))
    
    # Create summary table
    summary = []
    for name in selected_names:
        count = len(srcs[srcs['Group'] == name])
        summary.append({"Name": name, "Count": count})
    
    summary_df = pd.DataFrame(summary).sort_values("Count", ascending=False)
    display(summary_df)
else:
    display(HTML("""
    <div style="padding: 10px; background-color: #fff3e0; border-radius: 5px; border-left: 4px solid #FF9800;">
        <b>⚠️ No selections yet.</b> Click the button after selecting sources on the map.
    </div>
    """))

In [ ]:
# Ellipses require at least 2 data points per group.
# Split Sources into two tables called 'srcs' and 'onerow' for single sample sources

# Count how many times each group appears
counts = srcs['Group'].value_counts()

# DataFrame with groups that appear < 2 times
onesample = srcs[srcs['Group'].map(counts) < 2]

# DataFrame with sources that have >= 2 samples
srcs = srcs[srcs['Group'].map(counts) >= 2]

print('Ellipses will be created for the following Source Groups:')
print(srcs['Group'].value_counts())

print('Points will be shown for these Sources with <2 samples:')

if onesample is not None and not onesample.empty:
    print(onesample)
else:
    print('(All sources have sufficient samples for ellipses)')


In [ ]:
# This cell contains code for creating 1 s.d. confidence ellipses on biplots

def confidence_ellipse(x, y, n_std=1.96, size=100):   # Ellipses in Plotly
    """
    Get the covariance confidence ellipse of *x* and *y*.
    from https://gist.github.com/dpfoose/38ca2f5aee2aea175ecc6e599ca6e973

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.
    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.
    size : int
        Number of points defining the ellipse
    Returns
    -------
    String containing an SVG path for the ellipse

    References (H/T)
    ----------------
    https://matplotlib.org/3.1.1/gallery/statistics/confidence_ellipse.html
    https://community.plotly.com/t/arc-shape-with-path/7205/5
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    theta = np.linspace(0, 2 * np.pi, size)
    ellipse_coords = np.column_stack([ell_radius_x * np.cos(theta), ell_radius_y * np.sin(theta)])

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    x_scale = np.sqrt(cov[0, 0]) * n_std
    x_mean = np.mean(x)

    # calculating the stdandard deviation of y ...
    y_scale = np.sqrt(cov[1, 1]) * n_std
    y_mean = np.mean(y)

    translation_matrix = np.tile([x_mean, y_mean], (ellipse_coords.shape[0], 1))
    rotation_matrix = np.array([[np.cos(np.pi / 4), np.sin(np.pi / 4)],
                                [-np.sin(np.pi / 4), np.cos(np.pi / 4)]])
    scale_matrix = np.array([[x_scale, 0],
                            [0, y_scale]])
    ellipse_coords = ellipse_coords.dot(rotation_matrix).dot(scale_matrix) + translation_matrix

    path = f'M {ellipse_coords[0, 0]}, {ellipse_coords[0, 1]}'
    for k in range(1, len(ellipse_coords)):
        path += f'L{ellipse_coords[k, 0]}, {ellipse_coords[k, 1]}'
    path += ' Z'
    return path



In [ ]:
# Assign a color to each Source for consistency
name_to_color = {}

unique_groups = srcs['Group'].dropna().unique()  # drop NA just in case
colors = DEFAULT_PLOTLY_COLORS

# Cycle colors if more groups than colors
name_to_color = {name: colors[i % len(colors)] for i, name in enumerate(unique_groups)}

# 2️⃣ If you want a simple mapping to original name (optional)
unique_name_to_name = {name: name for name in unique_groups}

In [ ]:
# BIPLOT
# 
# To change the variables modify x_col and y_col here
# and Re-run the cell
print('Available elements to plot.')
print(srcs.columns)
print('To change elements modify x_col and y_col variables and re-run this cell:')

# Variables to plot
x_col = 'Sr'
y_col = 'Rb'
group = 'Group'

figsrcs = go.Figure()


# Add source ellipses
for g in srcs[group].unique():
    # Map group to color, default to gray if missing
    color = name_to_color.get(unique_name_to_name.get(g, ""), "gray")

    # Optional: add source points (uncomment lines below to show points for sources instead 
    # of just the ellipses)
    # figsrcs.add_trace(
    #     go.Scatter(
    #         x=srcs[srcs[group] == g][x_col],
    #         y=srcs[srcs[group] == g][y_col],
    #         name=g,
    #         mode='markers',
    #         marker=dict(symbol='x', size=4, color=color)
    #     )
    # )

    # Add confidence ellipse for the group
    figsrcs.add_shape(
        type='path',
        path=confidence_ellipse(
            srcs[srcs[group] == g][x_col],
            srcs[srcs[group] == g][y_col]
        ),
        line_color=color,
        name=g,
        showlegend=True
    )

# Add single source sample point
figsrcs.add_trace(
    go.Scatter(
        x=onesample[x_col],
        y=onesample[y_col],
        name="Source Sample",
        mode='markers',
        marker=dict(symbol='x', size=4, color='black'),
        text=onesample['Sample'],
        hovertemplate="Source: %{text}<br><extra></extra>",
        showlegend=True
    )
)

# Map Study sample groups to colors
study_colors = [
    name_to_color.get(unique_name_to_name.get(g, ""), "gray")
    for g in study[group]
]

# Add study samples colored by Group
figsrcs.add_trace(
    go.Scatter(
        x=study[x_col],
        y=study[y_col],
        name="Study Samples",
        mode='markers',
        text=study['Sample'],
        hovertemplate="Sample: %{text}<br><extra></extra>",
        showlegend=True,
        marker=dict(
            size=8,
            symbol='circle',
            color=study_colors
        )
    )
)
# ------------------------------
# Add confidence ellipses for study groups
# ------------------------------
for g in study[group].unique():
    mask = study[group] == g
    data_x = study.loc[mask, x_col]
    data_y = study.loc[mask, y_col]
    
    # Skip empty groups
    if len(data_x) < 2:
        continue
    
    color = name_to_color.get(g, "gray")
    
    figsrcs.add_shape(
        type='path',
        path=confidence_ellipse(data_x, data_y),
        line_color=color,
        line_width=2,
        name=f"{g} Study Ellipse",
        legendgroup=g,
        showlegend=True,
        opacity=0.5
    )

# Set layout with centered title
figsrcs.update_layout(
    title={
        'text': "Connecting Study Samples with known obsidian sources",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title=x_col,
    yaxis_title=y_col
)

for g in study[group].unique():
    figsrcs.add_trace(
        go.Scatter(
            x=[None], y=[None],
            mode="lines",
            line=dict(color=name_to_color.get(g, "gray")),
            name=f"Study {g}",
            showlegend=True
        )
    )

figsrcs.show(renderer="notebook")

### Biplot generated for visual source identification
Investigate biplot shown above. You may change the element variables at start of previous cell and re-run cell in order to view source ellipses and study samples using different elements on X and Y axes.
Proceed to Ternary diagram below to view three element variables at once.

In [ ]:
# TERNARY PLOT CODE

# --- Function to compute ellipse in 2D (fractions space) ---
def confidence_ellipse_points(x, y, n_std=1.96, size=100):
    """
    Return Nx2 array of (x,y) points for the confidence ellipse in 2D space.
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    mean_x = np.mean(x)
    mean_y = np.mean(y)

    eigvals, eigvecs = np.linalg.eigh(cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]

    theta = np.linspace(0, 2 * np.pi, size)
    ellipse = np.stack([np.cos(theta), np.sin(theta)], axis=1)
    scale = n_std * np.sqrt(eigvals)
    ellipse = ellipse @ np.diag(scale) @ eigvecs.T
    ellipse += np.array([mean_x, mean_y])

    return ellipse  # Nx2 array

# --- Normalize compositional data ---
def normalize_composition(df, cols):
    """
    Normalize rows so that the specified columns sum to 1
    """
    comp = df[cols].values
    row_sums = comp.sum(axis=1).reshape(-1,1)
    comp_norm = comp / row_sums
    return comp_norm

# Columns
cols = ['Rb','Sr','Zr']

# Normalize srcs and study
srcs_frac = normalize_composition(srcs, cols)
study_frac = normalize_composition(study, cols)

# --- Ternary Plot ---
fig = go.Figure()

# --- PLOT ELLIPSES ---
for g in srcs[group].unique():
    color = name_to_color.get(unique_name_to_name.get(g, ""), "gray")

    mask = srcs[group] == g
    rb, sr, zr = srcs_frac[mask].T

    ell = confidence_ellipse_points(rb, sr)

    eRb = ell[:,0]
    eSr = ell[:,1]
    eZr = 1 - eRb - eSr

    fig.add_trace(
        go.Scatterternary(
            a=eRb, b=eSr, c=eZr,
            mode='lines',
            line=dict(color=color),
            name=f"{g} Source",
            showlegend=True
        )
    )

# --- PLOT ONESAMPLE POINTS IF NON-EMPTY ---
if onesample is not None and not onesample.empty:
    onesample_frac = normalize_composition(onesample, cols)
    for g in onesample[group].unique():
        mask = onesample[group] == g
        data = onesample_frac[mask]
        color = name_to_color.get(unique_name_to_name.get(g, ""), "black")

        fig.add_trace(
            go.Scatterternary(
                a=data[:,0],
                b=data[:,1],
                c=data[:,2],
                mode='markers',
                name=f"Source Sample: {g}",
                marker=dict(symbol='x', size=6, color=color),
                text=onesample[mask]['Sample'],
                hovertemplate="Source: %{text}<extra></extra>"
            )
        )

# --- PLOT STUDY SAMPLES WITH DIFFERENT COLORS PER GROUP ---
for g in study[group].unique():
    mask = study[group] == g
    data = study_frac[mask]
    color = name_to_color.get(unique_name_to_name.get(g, ""), "gray")

    fig.add_trace(
        go.Scatterternary(
            a=data[:,0],
            b=data[:,1],
            c=data[:,2],
            mode='markers',
            name=f"Study: {g}",
            marker=dict(symbol='circle', size=8, color=color),
            text=study[mask]['Sample'],
            hovertemplate="Sample: %{text}<extra></extra>"
        )
    )

# --- Layout ---
fig.update_layout(
    ternary=dict(
        sum=1,
        aaxis=dict(title='Rb'),
        baxis=dict(title='Sr'),
        caxis=dict(title='Zr')
    ),
    title=dict(
        text="Ternary Plot with Obsidian Source Ellipses and Study Samples",
        x=0.5,
        xanchor='center',
        yanchor='top'
    )
)

fig.show()


### Ternary Plot
Examine relationship between samples and the ellipses representing known sources. Zoom into the Ternary diagram and change the variables shown in the previous cell if you wish to view other elements in the diagram.

### The analysis notebook has concluded.
In your data exploration mouse over the points in the biplot and ternary plot to view the unique ID numbers of those samples and note their relationship to ellipses for known obsidian sources.